<a href="https://colab.research.google.com/github/Busola181/NeuralNetworkBasedLanguageTranslationTool/blob/main/Neural_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

♈IMPORTING DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**♒

In [ ]:
API_KEY_PATH ="/content/drive/MyDrive/kaggle.json"

!mkdir -p ~/.kaggle
!cp $API_KEY_PATH ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d devicharith/language-translation-englishfrench

RESOURCES_PATH="/content/language-translation-englishfrench.zip"
!cp $RESOURCES_PATH.
!unzip /content/language-translation-englishfrench.zip -d /content/

Dataset URL: https://www.kaggle.com/datasets/devicharith/language-translation-englishfrench
License(s): CC0-1.0
100% 3.51M/3.51M [00:01<00:00, 3.86MB/s]
100% 3.51M/3.51M [00:01<00:00, 2.86MB/s]
cp: missing destination file operand after '.'
Try 'cp --help' for more information.
Archive:  /content/language-translation-englishfrench.zip
  inflating: /content/eng_-french.csv  


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Dataset
from collections import Counter
import re
import math
import nltk
import string
from torchsummary import summary
from unicodedata import normalize
from nltk.tokenize import word_tokenize



nltk.data.path.append('/usr/local/share/nltk_data')
nltk.download('wordnet', download_dir='/usr/local/share/nltk_data')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package wordnet to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# punctuations = set(filter(lambda char: char in string.punctuation, dataset_path))
# print(punctuations)

In [ ]:
# def load_doc(dataset_path, column_path):
#     text = df[column_path].str.cat(sep = '')
#     words = text.split()
#     print(words[:100])

# load_doc(dataset_path, 'English words/sentences')


In [ ]:
# def load_doc(dataset_path):
#     df = pd.read_csv(dataset_path)
#     return df

# def preprocess_text(text):
#     text = normalize('NFD', text).encode('ascii', 'ignore').decode('UTF-8')
#     text = text.lower()
#     text = re.sub(r"[_./-]", " ", text)
#     text = re.sub(r"[^a-zA-Z\s]", "", text)
#     tokens = word_tokenize(text)
#     return tokens

# def tokenize_pairs(df):
#     df['English tokens'] = df['English words/sentences'].apply(preprocess_text)
#     df['French tokens'] = df['French words/sentences'].apply(preprocess_text)
#     return df

# df = load_doc(dataset_path)

# df_tokenized = tokenize_pairs(df)

# print(df_tokenized[['English tokens', 'French tokens']].head(2))


In [ ]:
# def create_vocab(english_tokens, french_tokens, min_freq=1):

#     english_flat = [word for sentence in english_tokens for word in sentence]
#     french_flat = [word for sentence in french_tokens for word in sentence]

#     english_freq = Counter(english_flat)
#     french_freq = Counter(french_flat)

#     english_vocab = {word: idx + 4 for idx, (word, _) in enumerate(english_freq.items()) if english_freq[word] >= min_freq}
#     french_vocab = {word: idx + 4 for idx, (word, _) in enumerate(french_freq.items()) if french_freq[word] >= min_freq}

#     special_tokens = ['<PAD>', '<UNK>', '<BOS>', '<EOS>']
#     for token in special_tokens:
#         english_vocab[token] = len(english_vocab) + 1
#         french_vocab[token] = len(french_vocab) + 1

#     return english_vocab, french_vocab

# def text_to_tokens(text, vocab):
#     return [vocab.get(word, vocab['<UNK>']) for word in text]

# def pad_sequences(sequences, max_len, pad_token_idx):
#     return [seq + [pad_token_idx] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len] for seq in sequences]

# english_tokens = df_tokenized['English tokens'].tolist()
# french_tokens = df_tokenized['French tokens'].tolist()

# english_vocab, french_vocab = create_vocab(english_tokens, french_tokens)


# def text_to_tokens(text, vocab):
#     return [vocab.get(word, vocab['<UNK>']) for word in text]

# def pad_sequences(sequences, max_len, pad_token_idx):
#     return [seq + [pad_token_idx] * (max_len - len(seq)) if len(seq) < max_len else seq[:max_len] for seq in sequences]

# df_tokenized = tokenize_pairs(df)
# english_tokens = df_tokenized['English tokens'].tolist()
# french_tokens = df_tokenized['French tokens'].tolist()

# english_vocab, french_vocab = create_vocab(english_tokens, french_tokens)

# sample_english = df_tokenized['English tokens'].iloc[0]
# sample_french = df_tokenized['French tokens'].iloc[0]
# english_tokenized = text_to_tokens(sample_english, english_vocab)
# french_tokenized = text_to_tokens(sample_french, french_vocab)

# eng_input_tensor = torch.tensor(english_tokenized, dtype = torch.long)
# fren_target_tensor = torch.tensor(french_tokenized, dtype = torch.long)

# print(eng_input_tensor.shape)
# print(fren_target_tensor.shape)

# eng_input_tensor = sorted(set)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
dataset_path = '/content/eng_-french.csv'

def load_doc(dataset_path):
    df = pd.read_csv(dataset_path)
    return df

def preprocess_text(text):
    return tokenizer.encode(text, truncation = True, padding = "max_length", max_length = 100)

def tokenize_pairs(df):
    df['English tokens'] = df['English words/sentences'].apply([preprocess_text])
    df['French tokens'] = df['French words/sentences'].apply(preprocess_text)
    return df

df = load_doc(dataset_path)
df = tokenize_pairs(df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# Load and preprocess the dataset
df = load_doc(dataset_path)
df_tokenized = tokenize_pairs(df)  # Save the tokenized DataFrame

class TranslationDataset(Dataset):
    def __init__(self, english_tokens, french_tokens):
        self.english_tokens = english_tokens
        self.french_tokens = french_tokens

    def __len__(self):
        return len(self.english_tokens)

    def __getitem__(self, idx):
        english_sentence = self.english_tokens[idx]
        french_sentence = self.french_tokens[idx]
        return torch.tensor(english_sentence), torch.tensor(french_sentence)

# Convert tokenized columns to lists
english_tokens = df_tokenized['English tokens'].tolist()
french_tokens = df_tokenized['French tokens'].tolist()  # Fixed typo here

# Create the dataset
dataset = TranslationDataset(english_tokens, french_tokens)

# Split the dataset into train, test, and validation sets
train_size = int(len(dataset) * 0.8)
test_size = int(len(dataset) * 0.1)
val_size = len(dataset) - train_size - test_size

train_dataset, test_dataset, val_dataset = random_split(dataset, [train_size, test_size, val_size])

# Create DataLoaders
batch_size = 32  # Set your desired batch size
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

def create_vocab(english_tokens, french_tokens):
    english_vocab = set(token for tokens in english_tokens for token in tokens)
    french_vocab = set(token for tokens in french_tokens for token in tokens)
    return english_vocab, french_vocab

english_vocab, french_vocab = create_vocab(english_tokens, french_tokens)

In [ ]:
print(len(english_vocab))
#print(len(french_vocab))

12535


In [ ]:
num_encoder_layers = 8
num_decoder_layers = 8
batch_size = 32
mod_vec_dim = 512
scr_vocab_size = len(english_vocab)
tgt_vocab_size = len(french_vocab)
seq_len = 100
src_seq_len = 100
tgt_seq_len = 100
embed_size = 512
num_heads = 8
input_size = 32, 100, 14099
head_dim = embed_size // num_heads
hidden_dim = 2048
num_epochs = 10
lr = 0.001
momentum = 0.9
weight_decay = 1e-4

In [ ]:
print(f"target_vocab_size {tgt_vocab_size}, source_vocab_size :{scr_vocab_size}")

target_vocab_size 5777, source_vocab_size :12535


In [ ]:
# class TranslationDataset(Dataset):
#     def __init__(self, english_tokens, french_tokens, english_vocab, french_vocab):
#         self.english_tokens = english_tokens
#         self.french_tokens = french_tokens
#         self.english_vocab = english_vocab
#         self.french_vocab = french_vocab

#     def __len__(self):
#         return len(self.english_tokens)

#     def __getitem__(self, idx):
#         english_sentence = self.english_tokens[idx]
#         french_sentence = self.french_tokens[idx]

#         english_indices = text_to_tokens(english_sentence, self.english_vocab)
#         french_indices = text_to_tokens(french_sentence, self.french_vocab)

#         return torch.tensor(english_indices), torch.tensor(french_indices)

# def pad_sequences(sequences, max_len, pad_token_idx):
#     return [torch.cat((seq, torch.tensor([pad_token_idx] * (max_len - len(seq)))), dim=0) if len(seq) < max_len else seq[:max_len] for seq in sequences]


# def collate_fn(batch, pad_token_idx=0):
#     english_batch, french_batch = zip(*batch)

#     english_max_len = max([len(sentence) for sentence in english_batch])
#     french_max_len = max([len(sentence) for sentence in french_batch])

#     english_batch_padded = pad_sequences(english_batch, english_max_len, pad_token_idx)
#     french_batch_padded = pad_sequences(french_batch, french_max_len, pad_token_idx)

#     english_batch_padded = torch.stack(english_batch_padded)
#     french_batch_padded = torch.stack(french_batch_padded)

#     return english_batch_padded, french_batch_padded

# english_tokens = df_tokenized['English tokens'].tolist()
# french_tokens = df_tokenized['French tokens'].tolist()

# dataset = TranslationDataset(english_tokens, french_tokens, english_vocab, french_vocab)

# train_size = int(0.8 * len(dataset))
# val_size = int(0.1 * len(dataset))
# test_size = len(dataset) - train_size - val_size

# train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)


# for batch in train_dataloader:
#     english_batch, french_batch = batch
#     print(english_batch.shape)
#     print(french_batch.shape)
#     break



In [ ]:
class input_Embedding(nn.Module):
    def __init__(self, mod_vec_dim, vocab_size):
        super().__init__()
        self.mod_vec_dim = mod_vec_dim
        self.vocab_size = vocab_size
        self.embed = nn.Embedding(vocab_size, mod_vec_dim)

    def forward(self, x):
        return self.embed(x) * math.sqrt(self.mod_vec_dim)

class position_encoding(nn.Module):
    def __init__(self, mod_vec_dim, seq_len):
        super().__init__()
        self.mod_vec_dim = mod_vec_dim
        self.seq_len = seq_len
        position = torch.arange(0, seq_len, dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, mod_vec_dim, 2).float() * -math.log(10000)/ mod_vec_dim)
        pe = torch.zeros(seq_len, mod_vec_dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2]= torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

        self.dropout = nn.Dropout(p= 0.2)



    def forward(self, x):
        x = x + (self.pe[:, :x.size(1), :]).requires_grad_(False)
        return self.dropout(x)

# embed_dim == model_vec_dim
# For single head
class self_attention(nn.Module):
    def __init__(self, embed_size, head_size):
        super(self_attention, self).__init__()
        self.query = nn.Linear(embed_size, head_size, bias =False)
        self.key = nn.Linear(embed_size, head_size, bias =False)
        self.value = nn.Linear(embed_size, head_size, bias =False)
        # self.linear_proj = nn.Linear(embed_dim, head_size, bias=False)
        self.head_size = head_size
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        attention_scores = torch.matmul(query, key.transpose(-2, -1))/ math.sqrt(self.head_size)
        attention_probs = self.softmax(attention_scores)

        out = torch.matmul(attention_probs, value)
        out = self.dropout(out)

        return out


# For Multi-head

class Multihead_Attention(nn.Module):
    def __init__(self, embed_size, head_size, num_heads, mask=None):
        super(Multihead_Attention, self).__init__()
        self.embed_size = embed_size
        self.head_size = head_size
        self.num_head = num_heads
        self.head_dim = embed_size // num_heads

        assert self.head_dim * num_heads == embed_size

        self.w_q = nn.Linear(embed_size, embed_size)
        self.w_k = nn.Linear(embed_size, embed_size)
        self.w_v = nn.Linear(embed_size, embed_size)

        self.fc_out = nn.Linear(embed_size, embed_size)
        self.softmax = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(p=0.2)
        assert embed_size == num_heads * head_dim



    def forward(self, query, key, value, mask=None):
        seq_len = min(query.size(1), key.size(1), value.size(1))
        query = query[:, :seq_len, :]
        key = key[:, :seq_len, :]
        value = value[:, :seq_len, :]

        assert query.size(1) == key.size(1) == value.size(1)

        # # Print shapes before reshaping
        # print(f"query shape before reshaping: {query.shape}")
        # print(f"key shape before reshaping: {key.shape}")
        # print(f"value shape before reshaping: {value.shape}")

        N, seq_len, embed_dim = query.shape
        Q = self.w_q(query).view(N, seq_len, self.num_head, self.head_dim).transpose(1, 2)
        K = self.w_k(key).view(N, seq_len, self.num_head, self.head_dim).transpose(1, 2)
        V = self.w_v(value).view(N, seq_len, self.num_head, self.head_dim).transpose(1, 2)

         # Print shapes after reshaping
        # print(f"Q shape after reshaping: {Q.shape}")
        # print(f"K shape after reshaping: {K.shape}")
        # print(f"V shape after reshaping: {V.shape}")
        attention_scores = torch.matmul(Q, K.transpose(-2, -1))/ math.sqrt(self.head_dim)

        if mask is not None:
            if isinstance(mask, torch.Tensor):
                attention_scores = attention_scores.masked_fill(mask.bool(), float('-1e20'))

        attention_probs = self.softmax(attention_scores)
        out = torch.matmul(attention_probs, V)

        out = out.transpose(1, 2).contiguous().view(N, seq_len, self.embed_size)
        out = self.fc_out(out)

        return out

class feed_forward(nn.Module):
    def __init__(self, embed_size, hidden_dim, bias = False):
        super(feed_forward, self).__init__()
        self.fc1 = nn.Linear(embed_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = 0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
# Test Multihead_Attention
embed_size = 512
head_dim = 64
num_heads = 8
seq_len = 100
batch_size = 32

multihead_attention = Multihead_Attention(embed_size, head_dim, num_heads, mask = None)

query = torch.rand(batch_size, seq_len, embed_size)
key = torch.rand(batch_size, seq_len, embed_size)
value = torch.rand(batch_size, seq_len, embed_size)

output = multihead_attention(query, key, value)
print(f"Output shape: {output.shape}")


Output shape: torch.Size([32, 100, 512])


In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self,embed_size, num_heads, head_dim):
        super(TransformerEncoder, self).__init__()

        self.attention = Multihead_Attention(embed_size, head_dim, num_heads, mask= None)
        self.feed_forward = nn.Sequential(
             nn.Linear(embed_size, hidden_dim),
             nn.Linear(hidden_dim, embed_size),
             nn.Dropout(p=0.2),
             nn.ReLU())
        self.dropout= nn.Dropout(p= 0.2)

        self.Layer_norm1 = nn.LayerNorm(embed_size)
        self.Layer_norm2 = nn.LayerNorm(embed_size)

    def forward(self, x, src_mask = None):
        attention_out = self.attention(x, x, x, mask = None)

        out = self.Layer_norm1(x + self.dropout(attention_out))
        fc_out = self.feed_forward(out)

        out = self.Layer_norm2(x + self.dropout(fc_out))
        return(out)

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, embed_size, num_heads, hidden_dim, dropout):
        super(TransformerDecoder, self).__init__()

        self.masked_attention = Multihead_Attention(embed_size, head_dim, num_heads, mask= None)

        self.enc_dec_attention = Multihead_Attention(embed_size,head_dim, num_heads, mask=None)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, embed_size)
        )

        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.norm3 = nn.LayerNorm(embed_size)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask=None, tgt_mask=None):
        masked_attention_out = self.masked_attention(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(masked_attention_out))

        enc_dec_attention_out = self.enc_dec_attention(x, enc_out, enc_out, src_mask)
        x = self.norm2(x + self.dropout(enc_dec_attention_out))

        forward_out = self.feed_forward(x)
        out = self.norm3(x + self.dropout(forward_out))

        return out


In [ ]:
class TransformerModel(nn.Module):
    def __init__(self,scr_vocab_size, tgt_vocab_size, embed_size, num_heads, head_dim, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.scr_word_embedding = nn.Embedding(scr_vocab_size, embed_size)
        self.tgt_word_embedding = nn.Embedding(tgt_vocab_size, embed_size)

        self.positional_encoding = position_encoding(mod_vec_dim, seq_len)


        self.encoder_layers = nn.ModuleList([TransformerEncoder(embed_size, num_heads, head_dim)
        for _ in range(num_encoder_layers)]) # Model List comprehension
        self.decoder_layers = nn.ModuleList([TransformerDecoder(embed_size, num_heads, head_dim, dropout)
        for _ in range(num_decoder_layers)])

        # mapping decoder to my target
        self.fc_out = nn.Linear(embed_size, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, scr, tgt, src_mask = None, tgt_mask = None):
        scr = self.dropout(self.positional_encoding(self.scr_word_embedding(scr))).to(device)
        tgt = self.dropout(self.positional_encoding(self.tgt_word_embedding(tgt))).to(device)

        for encoder in self.encoder_layers:
            scr = encoder(scr, src_mask)
        for decoder in self.decoder_layers:
            tgt = decoder(tgt, scr, src_mask,tgt_mask)

        out = self.fc_out(tgt)
        return out


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = TransformerModel(scr_vocab_size, tgt_vocab_size, embed_size, num_heads, head_dim, dropout = 0.2).to(device)

scr = torch.randint(0, 12535, (batch_size, seq_len)).to(device)
tgt = torch.randint(0, 5777, (batch_size, seq_len)).to(device)
output = model(scr, tgt)
print(output.shape)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)
# summary(model, input_size = input_size)

torch.Size([32, 100, 5777])


In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
def train(model, train_dataloader, val_dataloader, optimizer, criterion, num_epochs, device):
    vocab_size = 5777
    scr_vocab_size = 12535

    for epoch in range(num_epochs):
        model.train()
        epoch_train_loss = 0
        correct_train = 0
        total_train = 0

        for batch_idx, (scr, tgt) in enumerate(train_dataloader):
            print(f"Batch {batch_idx}:")
            print(f"Source shape: {scr.shape}, Target shape: {tgt.shape}")
            print(f"Max tgt value: {tgt.max()}, Min tgt value: {tgt.min()}")

            # Move to device
            scr = scr.to(device)
            tgt = tgt.to(device)

            optimizer.zero_grad()

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            # Ensure target values are within valid range
            print(f"Max tgt_output before training: {tgt_output.max()}")
            assert tgt_output.max() < vocab_size, f"Target index out of range! Max: {tgt_output.max()}, Vocab size: {vocab_size}"
            assert tgt_output.min() >= 0, f"Negative target index found: {tgt_output.min()}"

            output = model(scr, tgt_input)
            output = output.reshape(-1, output.shape[-1])
            tgt_output = tgt_output.reshape(-1)

            loss = criterion(output, tgt_output)
            epoch_train_loss += loss.item()

            _, predicted = torch.max(output, dim=1)
            correct_train += (predicted == tgt_output).sum().item()
            total_train += tgt_output.size(0)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

        avg_train_loss = epoch_train_loss / len(train_dataloader)
        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")


# #         # Validation step (optional but important)
# #         model.eval()
# #         epoch_val_loss = 0
# #         correct_val = 0
# #         total_val = 0
# #         with torch.no_grad():
# #             for batch_idx, (src, tgt) in enumerate(val_dataloader):
# #                 # Truncate to the same length
# #                 max_len = min(src.size(1), tgt.size(1))
# #                 src = src[:, :max_len]
# #                 tgt = tgt[:, :max_len]

# #                 # Move to device
# #                 src = src.to(device)
# #                 tgt = tgt.to(device)

# #                 tgt_input = tgt[:, :-1]  # Exclude last token
# #                 tgt_output = tgt[:, 1:]  # Exclude first token

# #                 # Forward pass
# #                 output = model(src, tgt_input)
# #                 output = output.reshape(-1, output.shape[-1])
# #                 tgt_output = tgt_output.reshape(-1)

# #                 # Ensure valid target indices
# #                 assert tgt_output.max() < vocab_size, f"Target values exceed vocab size in validation: {tgt_output.max()} >= {vocab_size}"
# #                 assert tgt_output.min() >= 0, f"Target values are negative in validation: {tgt_output.min()} < 0"

# #                 # Compute loss
# #                 loss = criterion(output, tgt_output)
# #                 epoch_val_loss += loss.item()

# #                 # Compute validation accuracy
# #                 _, predicted = torch.max(output, dim=1)
# #                 correct_val += (predicted == tgt_output).sum().item()
# #                 total_val += tgt_output.size(0)

# #         avg_val_loss = epoch_val_loss / len(val_dataloader)
# #         val_accuracy = 100 * correct_val / total_val

# #         print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")



In [ ]:
print(f"Encoder output shape: {scr.shape}")
print(f"Decoder input shape: {tgt.shape}")


Encoder output shape: torch.Size([32, 100])
Decoder input shape: torch.Size([32, 100])


In [ ]:
print(f"model output shape: {output.shape}, target_vocab_size :{tgt_vocab_size}")

model output shape: torch.Size([32, 100, 5777]), target_vocab_size :5777


In [ ]:
train(model, train_dataloader, val_dataloader, optimizer, criterion, num_epochs, device)

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [ ]:
input_sentence = ['Hi there', 'how are you']
inputs = tokenizer(input_sentence, return_tensors='pt', padding=True)
translated = model.generate(**inputs, num_beams=5, max_length=40, early_stopping=True)

translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
print("translated Sentence", translated_text)

In [ ]:
!pip install sacrebleu
import sacrebleu
target_translation = [['Bonjour.'], [ 'comment allez-vous']]

bleu = sacrebleu.corpus_bleu(translated_text, target_translation)
print("BLeuscore:", bleu.score)

In [ ]:

def evaluate(model, test_dataloader, criterion, device='cpu'):
    model.eval()  # Set the model to evaluation mode
    epoch_test_loss = 0
    correct_test = 0
    total_test = 0

    with torch.no_grad():  # Disable gradient calculation
        for batch_idx, (src, tgt) in enumerate(test_dataloader):
            # Ensure indices are within bounds
            src = torch.clamp(src, max=scr_vocab_size - 1).to(device='cpu')
            tgt = torch.clamp(tgt, max=tgt_vocab_size - 1).to(device='cpu')

            max_len = min(src.size(1), tgt.size(1))
            src = src[:, :max_len]
            tgt = tgt[:, :max_len]

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            output = model(src, tgt_input)
            output = output.reshape(-1, output.shape[-1])
            tgt_output = tgt_output.reshape(-1)

            loss = criterion(output, tgt_output)
            epoch_test_loss += loss.item()

            _, predicted = torch.max(output, dim=1)
            correct_test += (predicted == tgt_output).sum().item()
            total_test += tgt_output.size(0)

    avg_test_loss = epoch_test_loss / len(test_dataloader)
    test_accuracy = 100 * correct_test / total_test

    print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

evaluate(model, test_dataloader, criterion, device)